# Driver

## Imports

In [1]:
import modin.pandas as pd
from transformer.DataAggregator import DataAggregator
from transformer.Dataformator import DataFormator
from transformer.ImputeMean import ImputeMean
from transformer.TrainTestSplit import TrainTestSplit
import utils
import numpy as np
import json
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.metrics import mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from configs.space import space

## Data Loading

In [2]:
sales = pd.read_csv("./data/sales_train_validation.csv")
calender =pd.read_csv("./data/calendar.csv")

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()

Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


## Data Transformation

In [3]:
data_aggregator = DataAggregator(sales)
aggregated_data = data_aggregator.aggregate(["store_id","dept_id"],"sum")
data_formator =DataFormator(aggregated_data,calender)
data = data_formator.format_data('store_id','dept_id')
impute_mean =ImputeMean(data)
data = impute_mean.replace_zero_with_mean()
tts = TrainTestSplit( data, test_size=0.3, random_state=0,shuffle=False)
X_train_, X_test_, y_train_, y_test_ = tts.split_data()

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


## Parameter Space for Xgboost data preparation
The link to papers is given below in the notebook, for The approch used in this model data preparation.

In [4]:
target_sequence_length = 1 #The forecasting horizon
test_size = 0.30
# for data preparation for xgboost
hyperparameters = {
    "in_length" : 1, # =target_sequence_length, forecasting horizon length
    "step_size" : 4, # window size
    }

## Training for best Parameters Model tuning

### Model Tuning

In [5]:
best ={}

for i in data.columns:
    y_train_data_ = pd.DataFrame()
    y_test_data_ = pd.DataFrame()
    y_train_data_[i] = y_train_[i]
    y_test_data_[i] = y_test_[i]
    
    x_train, y_train,x_test, y_test = utils.prepare_data_for_xgb(
                                    y_train_data_,y_test_data_,
                                    hyperparameters["in_length"],hyperparameters["step_size"],
                                    target_sequence_length
                                                            )
    def objective(params):
        model = xgb.XGBRegressor(**params)
        model = MultiOutputRegressor(model).fit(x_train, y_train)
        
        # train_forecasts = model.predict(x_train)
        test_forecasts = model.predict(x_test)
        mse = mean_absolute_error(y_test, test_forecasts)
        # print(f'Test MAE: { mse}')
        # print("Mean test data value: {}".format(np.mean(y_test)))
        return {'loss': mse, 'status': STATUS_OK}

    # optimize hyperparameters using Hyperopt's Tree-structured Parzen Estimator (TPE) algorithm
    print(f"Training .......\n model_{i} ")
    best["best_param_{0}".format(i)] = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100)

    print(best["best_param_{0}".format(i)])

Training .......
 model_CA_1_FOODS_1 
100%|██████████| 100/100 [00:19<00:00,  5.06trial/s, best loss: 52.586634815989676]
{'gamma': 0.5588210624767112, 'learning_rate': 0.05074997617448306, 'max_depth': 0, 'n_estimators': 64, 'reg_alpha': 0.1570738033857957, 'reg_lambda': 0.3811456627123333, 'subsample': 0.9558001899293613}


Training .......
 model_CA_1_FOODS_2 
100%|██████████| 100/100 [00:20<00:00,  4.88trial/s, best loss: 79.29041146231698]
{'gamma': 0.12388016989323263, 'learning_rate': 0.15542105913122262, 'max_depth': 0, 'n_estimators': 127, 'reg_alpha': 0.8712273858269363, 'reg_lambda': 0.5910378848614662, 'subsample': 0.6599834393809063}


Training .......
 model_CA_1_FOODS_3 
100%|██████████| 100/100 [00:19<00:00,  5.08trial/s, best loss: 334.1713662314248]
{'gamma': 0.36465572675570623, 'learning_rate': 0.029430824850158386, 'max_depth': 0, 'n_estimators': 14, 'reg_alpha': 0.7453984450528848, 'reg_lambda': 0.70165266394807, 'subsample': 0.1745655713417647}


Training .......
 model_CA_1_HOBBIES_1 
100%|██████████| 100/100 [00:19<00:00,  5.17trial/s, best loss: 86.95195700905539]
{'gamma': 0.7556096198694703, 'learning_rate': 0.03747273181330879, 'max_depth': 0, 'n_estimators': 55, 'reg_alpha': 0.09641772946373879, 'reg_lambda': 0.9981059636634522, 'subsample': 0.10153131495183741}


Training .......
 model_CA_1_HOBBIES_2 
100%|██████████| 100/100 [00:17<00:00,  5.71trial/s, best loss: 17.002576974722054]
{'gamma': 0.3474895008233909, 'learning_rate': 0.14389199385035126, 'max_depth': 0, 'n_estimators': 171, 'reg_alpha': 0.6345637879092634, 'reg_lambda': 0.009768594613932202, 'subsample': 0.561030415398034}


...

Training .......
 model_WI_3_FOODS_3 
100%|██████████| 100/100 [00:23<00:00,  4.20trial/s, best loss: 272.56447873415647]
{'gamma': 0.5697316181384277, 'learning_rate': 0.028045941420302697, 'max_depth': 5, 'n_estimators': 5, 'reg_alpha': 0.16181549596556433, 'reg_lambda': 0.6838399371848547, 'subsample': 0.2392566816512926}


Training .......
 model_WI_3_HOBBIES_1 
100%|██████████| 100/100 [00:22<00:00,  4.37trial/s, best loss: 45.282028625061464]
{'gamma': 0.1602469272135567, 'learning_rate': 0.01875816639394134, 'max_depth': 0, 'n_estimators': 105, 'reg_alpha': 0.29741130568266017, 'reg_lambda': 0.3481557780155611, 'subsample': 0.5277959382129103}


Training .......
 model_WI_3_HOBBIES_2 
100%|██████████| 100/100 [00:20<00:00,  4.86trial/s, best loss: 8.651480681412703]
{'gamma': 0.3200786419139773, 'learning_rate': 0.09356973620601616, 'max_depth': 0, 'n_estimators': 6, 'reg_alpha': 0.28862362268090747, 'reg_lambda': 0.6419828093066774, 'subsample': 0.643712103850159}


Training .......
 model_WI_3_HOUSEHOLD_1 
100%|██████████| 100/100 [00:24<00:00,  4.13trial/s, best loss: 96.38408890304032]
{'gamma': 0.06366564696466426, 'learning_rate': 0.025081050996349802, 'max_depth': 0, 'n_estimators': 93, 'reg_alpha': 0.23600009987152074, 'reg_lambda': 0.7356317991811427, 'subsample': 0.5940076822488978}


Training .......
 model_WI_3_HOUSEHOLD_2 
100%|██████████| 100/100 [00:22<00:00,  4.50trial/s, best loss: 24.413331892106918]
{'gamma': 0.1790459984105559, 'learning_rate': 0.03075873982033497, 'max_depth': 0, 'n_estimators': 98, 'reg_alpha': 0.4626942425458335, 'reg_lambda': 0.5325705383823421, 'subsample': 0.48234789966913555}


### Save Best parameters

In [6]:
utils.save_to_json('temp',best,'w')

## Training with best parameters Model Training

### Reading Best Parameters

In [5]:
best =utils.read_from_json('xgb_best_params',mode="r")
best.keys()

dict_keys(['best_param_CA_1_FOODS_1', 'best_param_CA_1_FOODS_2', 'best_param_CA_1_FOODS_3', 'best_param_CA_1_HOBBIES_1', 'best_param_CA_1_HOBBIES_2', 'best_param_CA_1_HOUSEHOLD_1', 'best_param_CA_1_HOUSEHOLD_2', 'best_param_CA_2_FOODS_1', 'best_param_CA_2_FOODS_2', 'best_param_CA_2_FOODS_3', 'best_param_CA_2_HOBBIES_1', 'best_param_CA_2_HOBBIES_2', 'best_param_CA_2_HOUSEHOLD_1', 'best_param_CA_2_HOUSEHOLD_2', 'best_param_CA_3_FOODS_1', 'best_param_CA_3_FOODS_2', 'best_param_CA_3_FOODS_3', 'best_param_CA_3_HOBBIES_1', 'best_param_CA_3_HOBBIES_2', 'best_param_CA_3_HOUSEHOLD_1', 'best_param_CA_3_HOUSEHOLD_2', 'best_param_CA_4_FOODS_1', 'best_param_CA_4_FOODS_2', 'best_param_CA_4_FOODS_3', 'best_param_CA_4_HOBBIES_1', 'best_param_CA_4_HOBBIES_2', 'best_param_CA_4_HOUSEHOLD_1', 'best_param_CA_4_HOUSEHOLD_2', 'best_param_TX_1_FOODS_1', 'best_param_TX_1_FOODS_2', 'best_param_TX_1_FOODS_3', 'best_param_TX_1_HOBBIES_1', 'best_param_TX_1_HOBBIES_2', 'best_param_TX_1_HOUSEHOLD_1', 'best_param_TX_

### Model Training

In [6]:
trained_model={}

for i in data.columns:
    y_train_data_ = pd.DataFrame()
    y_test_data_ = pd.DataFrame()
    y_train_data_[i] = y_train_[i]
    y_test_data_[i] = y_test_[i]
    
    x_train, y_train,x_test, y_test = utils.prepare_data_for_xgb(
                                    y_train_data_,y_test_data_,
                                    hyperparameters["in_length"],hyperparameters["step_size"],
                                    target_sequence_length
                                                            )
    
    
    model = xgb.XGBRegressor(
        gamma= best[f"best_param_{i}"]['gamma'],
        learning_rate= best[f"best_param_{i}"]['learning_rate'],
        max_depth= best[f"best_param_{i}"]['max_depth'],
        n_estimators= best[f"best_param_{i}"]['n_estimators'],
        reg_alpha= best[f"best_param_{i}"]['reg_alpha'],
        reg_lambda= best[f"best_param_{i}"]['reg_lambda'],
        subsample= best[f"best_param_{i}"]['subsample']
        )

    trained_model["model_{0}".format(i)] = MultiOutputRegressor(model).fit(x_train, y_train)
    print(f"trained_model   :{i}")
    # train_forecasts = trained_model["model_{0}".format(i)].predict(x_train)
    test_forecasts = trained_model["model_{0}".format(i)].predict(x_test)
    test_mae = mean_absolute_error(y_test, test_forecasts)
    print(f'Test MAE: { test_mae}')
    # print("Mean test data value: {}".format(np.mean(y_test)))

trained_model   :CA_1_FOODS_1
Test MAE: 78.9238757153491


trained_model   :CA_1_FOODS_2
Test MAE: 84.73878446992461


trained_model   :CA_1_FOODS_3
Test MAE: 332.80848960609706


trained_model   :CA_1_HOBBIES_1
Test MAE: 109.77147498497597


trained_model   :CA_1_HOBBIES_2
Test MAE: 31.946973947378304


...

trained_model   :WI_3_FOODS_3
Test MAE: 300.9999368307474


trained_model   :WI_3_HOBBIES_1
Test MAE: 132.87426784488704


trained_model   :WI_3_HOBBIES_2
Test MAE: 9.265095810790163


trained_model   :WI_3_HOUSEHOLD_1
Test MAE: 304.84417319131063


trained_model   :WI_3_HOUSEHOLD_2
Test MAE: 29.56210540558075


In [7]:
model = trained_model['model_CA_1_FOODS_1']
model

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None,
                                            gamma=0.053543859402459315,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.43304389079434624,
                             

## Generating forecast

#### Refrence of papers for the used approch: 
https://arxiv.org/abs/1603.02754  &    
https://arxiv.org/abs/2101.02118
#### Refrence to site for data preparing steps:
https://towardsdatascience.com/multi-step-time-series-forecasting-with-xgboost-65d6820bec39

# Pipelines

In [5]:
import modin.pandas as pd
from transformer.DataAggregator import DataAggregator
from transformer.Dataformator import DataFormator
from transformer.ImputeMean import ImputeMean
from transformer.TrainTestSplit import TrainTestSplit
import utils
import numpy as np
import json
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.metrics import mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from configs.space import space

In [17]:
data = pd.read_csv('./data/store_dept_sales.csv',index_col='Unnamed: 0')

In [20]:
data.head()

,CA_1_FOODS_1,CA_1_FOODS_2,CA_1_FOODS_3,CA_1_HOBBIES_1,CA_1_HOBBIES_2,CA_1_HOUSEHOLD_1,CA_1_HOUSEHOLD_2,CA_2_FOODS_1,CA_2_FOODS_2,CA_2_FOODS_3,...,WI_2_HOBBIES_2,WI_2_HOUSEHOLD_1,WI_2_HOUSEHOLD_2,WI_3_FOODS_1,WI_3_FOODS_2,WI_3_FOODS_3,WI_3_HOBBIES_1,WI_3_HOBBIES_2,WI_3_HOUSEHOLD_1,WI_3_HOUSEHOLD_2
2011-01-29,297,674,2268,528,28,361,181,406,212,1575,...,10,353,98,152,583,2293,256,22,584,148
2011-01-30,284,655,2198,489,9,350,170,408,227,1286,...,7,268,94,138,585,2383,342,14,541,195
2011-01-31,214,396,1398,409,6,279,114,238,138,913,...,18,250,69,127,575,1841,228,20,420,106
2011-02-01,175,476,1607,383,9,278,123,240,174,1126,...,15,305,80,98,533,1965,183,11,327,94
2011-02-02,182,354,1496,263,5,195,135,220,102,956,...,1,118,32,87,340,1427,70,4,151,53
2011-02-03,191,486,1730,453,14,256,146,263,103,1128,...,6,259,107,170,853,2760,285,8,392,122
2011-02-04,224,554,1915,339,7,268,143,304,140,1341,...,10,318,100,136,612,2727,357,3,492,159


In [19]:
tts = TrainTestSplit( data, test_size=0.3, random_state=0,shuffle=False)
X_train_, X_test_, y_train_, y_test_ = tts.split_data()

In [27]:
from sklearn.impute import SimpleImputer
from transformerExpr.ClipNegValues import ClipNegValues
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:

p1 = Pipeline([
    ('Clip neg values', ClipNegValues),
    ('impute mean values', )
])